1. Weź geny V i J aminokwasów.
2. Cross join (każdy z każdym) per locus.
3. Ponumerować - celem jest wygapowanie do IMGT.
4. Generate HMMER profiles (check anarci scripts) per locus.
5. Inject hmmer profiles files into anarci.
6. Inject gene files into anarci.
7. ANARCI wrapper aligns to all profiles - set param to align only to H, K, L.

In [8]:
from Bio import SeqIO
from csv import reader
from riot_na.config import GENE_DB_DIR
from riot_na.data.model import Locus, Organism
from typing import Literal
from typing import IO
from notebooks.dict_merge import dict_merge
from pathlib import Path
import subprocess

In [10]:
GAP = "-"


def read_imgt_mappings(organism: Organism) -> dict[str, str]:
    with open(GENE_DB_DIR / "scheme_mappings" / organism.value / "imgt" / "scheme_mapping.csv") as file:
        return {gene_id: sequence for gene_id, sequence in reader(file)}


def read_v_genes(organism: Organism, locus: Locus, aa_gene_dir: str) -> dict[str, str]:
    return {
        record.id: str(record.seq)
        for record in SeqIO.parse(GENE_DB_DIR / "gene_db" / aa_gene_dir / "v_genes" / f"{organism}.fasta", "fasta")
        if record.id.startswith(locus.value.upper())
    }


def read_j_genes(organism: Organism, locus: Locus, aa_gene_dir: str) -> dict[str, str]:
    return {
        record.id: str(record.seq)
        for record in SeqIO.parse(
            GENE_DB_DIR / "gene_db" / aa_gene_dir / "j_genes" / organism.value / f"{locus}.fasta", "fasta"
        )
    }


def gap_sequence(sequence, mapping):
    sequence_iter = iter(sequence)
    return "".join(next(sequence_iter) if op == "M" else GAP for op in mapping)


def gap_sequences(genes: dict[str, str], imgt_mappings: dict[str, str]) -> dict[str, str]:
    return {gene_id: gap_sequence(sequence, imgt_mappings[gene_id]) for gene_id, sequence in genes.items()}


def gap_sequence_to_128(gapped_sequence: str, gene: Literal["v", "j"]) -> str:
    gaps = (128 - len(gapped_sequence)) * GAP
    match gene:
        case "v":
            return f"{gapped_sequence}{gaps}"
        case "j":
            return f"{gaps}{gapped_sequence}"


V_GENE_TRIM_POSITIONS = {
    Organism.HOMO_SAPIENS: {Locus.IGH: 107, Locus.IGK: 108, Locus.IGL: 108},
    Organism.MUS_MUSCULUS: {Locus.IGH: 108, Locus.IGK: 108, Locus.IGL: 108},
}


def cross_join(v_genes: dict[str, str], j_genes: dict[str, str], organism: Organism) -> dict[str, str]:
    cross_joined = {}
    for v_gene_id, v_gene_sequence in v_genes.items():
        for j_gene_id, j_gene_sequence in j_genes.items():
            gene_id = f"{organism.name.lower().capitalize()}_{v_gene_id}_{j_gene_id}"
            v_gene_sequence_trimmed = v_gene_sequence[: V_GENE_TRIM_POSITIONS[organism][locus]]
            gaps = (128 - len(v_gene_sequence_trimmed) - len(j_gene_sequence)) * GAP
            sequence = f"{v_gene_sequence_trimmed}{gaps}{j_gene_sequence}"
            assert len(sequence) == 128, len(sequence)
            cross_joined[gene_id] = sequence
    return cross_joined


def write_genes_to_stockholm(fd: IO[str], organism: Organism, locus: Locus, cross_joined_genes: dict[str, str]):
    fd.write("# STOCKHOLM 1.0\n")
    fd.write(f"#=GF ID {organism}_{locus.name[-1]}\n")
    for gene_id, sequence in cross_joined_genes.items():
        fd.write(f"{gene_id}\t{sequence}\n")
    fd.write(f"#=GC RF\t{'x'*128}\n")
    fd.write("//\n")


def get_all_germlines_dict(
    v_genes: dict[str, str],
    j_genes: dict[str, str],
    organism: Organism,
    locus: Locus,
):
    """
    Return genes in 'all_germlines.py' dict-of-dicts format, required by ANARCI.
    The format is {gene (V|J): {locus (H|K|L): {organism (human|mouse): {gene_id: sequence}}}}
    The sequence must be gapped to len of 128.
    """
    return {
        "V": {
            locus.name[-1]: {
                organism.value: {
                    gene_id: gap_sequence_to_128(sequence, gene="v") for gene_id, sequence in v_genes.items()
                }
            }
        },
        "J": {
            locus.name[-1]: {
                organism.value: {
                    gene_id: gap_sequence_to_128(sequence, gene="j") for gene_id, sequence in j_genes.items()
                }
            }
        },
    }


OUTPUT_DIR = Path().absolute().parent / "databases" / "anarci"
OUTPUT_DIR.mkdir(exist_ok=True, parents=True)

all_germlines = {}
with (
    open(OUTPUT_DIR / "ALL.stockholm", "w") as stockholm_fd,
    open(OUTPUT_DIR / "germlines.py", "w") as all_germlines_py_fd,
):

    for organism in Organism:
        imgt_mappings = read_imgt_mappings(organism)

        for locus in Locus:
            v_genes = read_v_genes(organism, locus, "aa_genes_first_allele")
            v_genes_gapped = gap_sequences(v_genes, imgt_mappings)
            j_genes = read_j_genes(organism, locus, "aa_genes_first_allele")
            j_genes_gapped = gap_sequences(j_genes, imgt_mappings)
            cross_joined_genes = cross_join(v_genes, j_genes, organism)
            write_genes_to_stockholm(stockholm_fd, organism, locus, cross_joined_genes)

            v_genes = read_v_genes(organism, locus, "aa_genes_deduplicated")
            j_genes = read_j_genes(organism, locus, "aa_genes_deduplicated")
            dict_merge(all_germlines, get_all_germlines_dict(v_genes, j_genes, organism, locus))

    all_germlines_py_fd.write(f"all_germlines={all_germlines}")


HMMS_DIR = OUTPUT_DIR / "HMMs"
HMMS_DIR.mkdir(exist_ok=True)

subprocess.run(["hmmbuild", "--hand", HMMS_DIR / "ALL.hmm", OUTPUT_DIR / "ALL.stockholm"])
subprocess.run(["hmmpress", "-f", HMMS_DIR / "ALL.hmm"])

# hmmbuild :: profile HMM construction from multiple sequence alignments
# HMMER 3.3.2 (Nov 2020); http://hmmer.org/
# Copyright (C) 2020 Howard Hughes Medical Institute.
# Freely distributed under the BSD open source license.
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
# input alignment file:             /home/bartosz/Documents/riot/databases/anarci/ALL.stockholm
# output HMM file:                  /home/bartosz/Documents/riot/databases/anarci/HMMs/ALL.hmm
# model architecture construction:  hand-specified by RF annotation
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

# idx name                  nseq  alen  mlen eff_nseq re/pos description
#---- -------------------- ----- ----- ----- -------- ------ -----------
1     human_H                354   128   128     1.74  0.590 
2     human_L                259   128   128     2.67  0.590 
3     human_K                205   128   128     1.97  0.590 
4     mouse_H               3558 

CompletedProcess(args=['hmmpress', '-f', PosixPath('/home/bartosz/Documents/riot/databases/anarci/HMMs/ALL.hmm')], returncode=0)